### Streaming Data from Kafka

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, IntegerType, BooleanType

In [5]:
spark = SparkSession.builder \
    .appName("KafkaSparkConsumer") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/kafka_checkpoint") \
    .config("spark.jars.packages", "io.confluent:kafka-spark_2.12:5.5.0") \
    .getOrCreate()

25/03/09 00:09:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [15]:
channel_schema = StructType() \
    .add("channel_id", StringType()) \
    .add("title", StringType()) \
    .add("description", StringType()) \
    .add("custom_url", StringType()) \
    .add("published_at", StringType()) \
    .add("country", StringType()) \
    .add("subscriber_count", IntegerType()) \
    .add("view_count", IntegerType()) \
    .add("video_count", IntegerType()) \
    .add("hidden_subscriber_count", BooleanType()) \
    .add("high_thumbnail", StringType())

In [16]:
video_schema = StructType() \
    .add("video_id", StringType()) \
    .add("title", StringType()) \
    .add("description", StringType()) \
    .add("description_summary", StringType()) \
    .add("channel_id", StringType()) \
    .add("channel_title", StringType()) \
    .add("published_at", StringType()) \
    .add("published_year", StringType()) \
    .add("view_count", IntegerType()) \
    .add("like_count", IntegerType()) \
    .add("comment_count", IntegerType()) \
    .add("favorite_count", IntegerType()) \
    .add("engagement_ratio", IntegerType()) \
    .add("likes_per_view", IntegerType()) \
    .add("comments_per_view", IntegerType()) \
    .add("thumbnail_url", StringType()) \
    .add("thumbnail_width", IntegerType()) \
    .add("thumbnail_height", IntegerType()) \
    .add("duration", StringType()) \
    .add("definition", StringType()) \
    .add("caption", BooleanType()) \
    .add("licensed_content", BooleanType()) \
    .add("tags", StringType()) \
    .add("tag_count", IntegerType()) \
    .add("category_id", StringType()) \
    .add("live_broadcast_content", StringType()) \
    .add("default_language", StringType()) \
    .add("default_audio_language", StringType()) \
    .add("privacy_status", StringType()) \
    .add("upload_status", StringType()) \
    .add("embeddable", BooleanType()) \
    .add("made_for_kids", BooleanType()) \
    .add("title_length", IntegerType()) \
    .add("description_length", IntegerType()) \
    .add("has_hashtags", BooleanType())

In [17]:
comment_schema = StructType() \
    .add("comment_id", StringType()) \
    .add("video_id", StringType()) \
    .add("author", StringType()) \
    .add("content", StringType()) \
    .add("published_at", StringType())

In [18]:
captions_schema = StructType() \
    .add("video_id", StringType()) \
    .add("caption_id", StringType()) \
    .add("language", StringType()) \
    .add("caption_text", StringType())

In [19]:
kafka_video_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "youtube_video_info") \
    .option("startingOffsets", "earliest") \
    .load()

kafka_channel_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "youtube_channel_info") \
    .option("startingOffsets", "earliest") \
    .load()

kafka_comment_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "youtube_comments") \
    .option("startingOffsets", "earliest") \
    .load()

kafka_captions_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "youtube_captions") \
    .option("startingOffsets", "earliest") \
    .load()

In [20]:
video_parsed_df = kafka_video_df \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), video_schema).alias("data")) \
    .select("data.*")

channel_parsed_df = kafka_channel_df \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), channel_schema).alias("data")) \
    .select("data.*")

comment_parsed_df = kafka_comment_df \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), comment_schema).alias("data")) \
    .select("data.*")

captions_parsed_df = kafka_captions_df \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), captions_schema).alias("data")) \
    .select("data.*")

25/03/09 00:24:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [22]:
video_parsed_df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- description_summary: string (nullable = true)
 |-- channel_id: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- published_at: string (nullable = true)
 |-- published_year: string (nullable = true)
 |-- view_count: integer (nullable = true)
 |-- like_count: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- engagement_ratio: integer (nullable = true)
 |-- likes_per_view: integer (nullable = true)
 |-- comments_per_view: integer (nullable = true)
 |-- thumbnail_url: string (nullable = true)
 |-- thumbnail_width: integer (nullable = true)
 |-- thumbnail_height: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- definition: string (nullable = true)
 |-- caption: boolean (nullable = true)
 |-- licensed_content: boolean (nullable = true)
 |-- tags

In [23]:
channel_parsed_df.printSchema()

root
 |-- channel_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- custom_url: string (nullable = true)
 |-- published_at: string (nullable = true)
 |-- country: string (nullable = true)
 |-- subscriber_count: integer (nullable = true)
 |-- view_count: integer (nullable = true)
 |-- video_count: integer (nullable = true)
 |-- hidden_subscriber_count: boolean (nullable = true)
 |-- high_thumbnail: string (nullable = true)



In [24]:
comment_parsed_df.printSchema()

root
 |-- comment_id: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)
 |-- published_at: string (nullable = true)



In [25]:
captions_parsed_df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- caption_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- caption_text: string (nullable = true)



In [26]:
video_query = video_parsed_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Await termination for 10 seconds
video_query.awaitTermination(10)

25/03/09 00:25:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/09 00:25:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+----------+----------+-------------+--------------+----------------+--------------+-----------------+--------------------+---------------+----------------+--------+----------+-------+----------------+--------------------+---------+-----------+----------------------+----------------+----------------------+--------------+-------------+----------+-------------+------------+------------------+------------+
|   video_id|               title|         description| description_summary|          channel_id|       channel_title|        published_at|published_year|view_count|like_count|comment_count|favorite_count|engagement_ratio|likes_per_view|comments_per_view|       thumbnail_url|thumbnail_width|thumbnail_height|duration|definition|caption|licens

False

In [27]:

channel_query = channel_parsed_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

channel_query.awaitTermination(10)

25/03/09 00:26:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/09 00:26:13 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+--------------------+--------------------+--------------------+--------------------+-------+----------------+----------+-----------+-----------------------+--------------------+
|          channel_id|               title|         description|          custom_url|        published_at|country|subscriber_count|view_count|video_count|hidden_subscriber_count|      high_thumbnail|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------+----------------+----------+-----------+-----------------------+--------------------+
|UCY1kMZp36IQSyNx_...|          Mark Rober|Former NASA engin...|          @markrober|2011-10-20T06:17:58Z|     US|        65000000|      NULL|        199|                  false|https://yt3.ggpht...|
|UCHnyfMqiRRG1u-2M...|          Veritasium|An element of tru...|         @veritasium|2010-07-21T07:18:0

False

In [30]:
# comment_query = comment_parsed_df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()

# comment_query.awaitTermination(10)

In [31]:
# captions_query = captions_parsed_df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()

# captions_query.awaitTermination(10)